In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r ../PETsARD/requirements.txt

import PETsARD

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.5,<3.8; 0.0.1.dev0 Requires-Python >=3.5,<3.8; 0.0.1.dev1 Requires-Python >=3.5,<3.8; 0.0.1.dev2 Requires-Python >=3.5,<3.8; 0.0.2 Requires-Python >=3.5,<3.9; 0.0.2.dev0 Requires-Python >=3.5,<3.8; 0.0.2.dev1 Requires-Python >=3.5,<3.9; 0.0.3 Requires-Python >=3.6,<3.9; 0.0.3.dev0 Requires-Python >=3.6,<3.9; 0.0.3.dev1 Requires-Python >=3.6,<3.9; 0.0.4 Requires-Python >=3.6,<3.9; 0.0.4.dev0 Requires-Python >=3.6,<3.9; 0.1.0 Requires-Python >=3.6,<3.9; 0.1.0.dev0 Requires-Python >=3.6,<3.9; 0.1.0.dev1 Requires-Python >=3.6,<3.9; 0.1.0.dev2 Requires-Python >=3.6,<3.9; 0.1.1 Requires-Python >=3.6,<3.9; 0.1.1.dev0 Requires-Python >=3.6,<3.9; 0.1.2 Requires-Python >=3.6,<3.9; 0.1.2.dev0 Requires-Python >=3.6,<3.9; 0.1.2.dev1 Requires-Python >=3.6,<3.9; 0.1.2.dev2 Requires-Python >=3.6,<3.9; 0.1.3 Requires-Python >=3.6,<3.9; 0.1.3.dev0 Requires-Python >=3.6,<3.9; 0.10.0 Requires-Python >=

In [2]:
import sdmetrics
sdmetrics.__version__

'0.13.0'

In [3]:
loader = PETsARD.Loader(
    filepath='../[sunset]/data/[Adt Income] adult.csv',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]
    }
)

splitter = PETsARD.Splitter(
    data=loader.data,
    num_samples=30,
    train_split_ratio=0.8
)

preproc = PETsARD.Preprocessor(
    data=splitter.data[1]['train'],
    missing=True,
    missing_method='drop',
    missing_columns=None,
    outlier=True,
    outlier_method='IQR',
    outlier_columns={'ignore': ['hours-per-week']},
    encoding=True,
    encoding_method='Label',
    encoding_columns=None,
    scaling=True,
    scaling_method='Standard',
    scaling_columns={'focus': 'fnlwgt'}
)

synthesizer = PETsARD.Synthesizer(
    data=preproc.data,
    synthesizing_method='sdv-singletable-gaussiancoupula'
)
synthesizer.fit_sample()

postproc = PETsARD.Postprocessor(
    data=synthesizer.data_syn,
    missingist = getattr(preproc ,'missingist' ,None),
    encoder=getattr(preproc, 'encoder', None),
    scaler=getattr(preproc, 'scaler', None)
)

postproc.data.head(6)

Preprocessor - Outlierist (IQR): Dropped   214 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3044 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1739 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   235 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  1041 rows on fnlwgt         . Kept [-64226.75, 420619.25] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6058 in 36178 rows.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (L

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,60.0,Private,202502.285408,Doctorate,10.0,Divorced,Priv-house-serv,Not-in-family,White,Male,0.0,0.0,40.0,Yugoslavia,<=50K
1,21.0,Private,142509.128124,Prof-school,10.0,Married-civ-spouse,Transport-moving,Not-in-family,White,Male,0.0,0.0,55.0,Vietnam,<=50K
2,27.0,Private,46194.976976,Bachelors,8.0,Never-married,Sales,Own-child,White,Female,0.0,0.0,34.0,Outlying-US(Guam-USVI-etc),<=50K
3,54.0,Private,226876.364196,Doctorate,11.0,Married-civ-spouse,Handlers-cleaners,Own-child,White,Male,0.0,0.0,46.0,Vietnam,<=50K
4,39.0,Private,224619.275947,Prof-school,8.0,Never-married,Armed-Forces,Wife,White,Female,0.0,0.0,44.0,Laos,<=50K
5,48.0,Self-emp-not-inc,194175.841062,Assoc-acdm,13.0,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0.0,0.0,45.0,South,>50K


In [4]:
# https://docs.sdv.dev/sdmetrics/
# https://github.com/sdv-dev/SDMetrics
from sdv.metadata import SingleTableMetadata
import pprint

real_data_metadata = SingleTableMetadata()
real_data_metadata.detect_from_dataframe(splitter.data[1]['train'])
real_data_metadata = real_data_metadata.to_dict()

pprint.pprint(real_data_metadata)


{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'age': {'sdtype': 'numerical'},
             'capital-gain': {'sdtype': 'numerical'},
             'capital-loss': {'sdtype': 'numerical'},
             'education': {'sdtype': 'categorical'},
             'educational-num': {'sdtype': 'numerical'},
             'fnlwgt': {'sdtype': 'numerical'},
             'gender': {'sdtype': 'categorical'},
             'hours-per-week': {'sdtype': 'numerical'},
             'income': {'sdtype': 'categorical'},
             'marital-status': {'sdtype': 'categorical'},
             'native-country': {'sdtype': 'categorical'},
             'occupation': {'sdtype': 'categorical'},
             'race': {'sdtype': 'categorical'},
             'relationship': {'sdtype': 'categorical'},
             'workclass': {'sdtype': 'categorical'}}}


In [5]:
from sdmetrics.reports.single_table import QualityReport

qlt_report = QualityReport()
qlt_report.generate(
    real_data = splitter.data[1]['train'],
    synthetic_data = postproc.data,
    metadata = real_data_metadata
)
print(f"\n\n")
print(f"get_score: {qlt_report.get_score()}")
print(f"\n\n")
print(qlt_report.get_properties())
print(f"\n\n")
print(qlt_report.get_details(property_name='Column Shapes'))
print(f"\n\n")
print(qlt_report.get_details(property_name='Column Pair Trends'))

Generating report ...
(1/2) Evaluating Column Shapes: :   0%|          | 0/15 [00:00<?, ?it/s]

(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:05<00:00, 19.68it/s]

Overall Score: 66.48%

Properties:
- Column Shapes: 84.58%
- Column Pair Trends: 48.39%



get_score: 0.6648355465503876



             Property     Score
0       Column Shapes  0.845816
1  Column Pair Trends  0.483855



             Column        Metric     Score
0               age  KSComplement  0.969232
1         workclass  TVComplement  0.991412
2            fnlwgt  KSComplement  0.952049
3         education  TVComplement  0.551716
4   educational-num  KSComplement  0.867366
5    marital-status  TVComplement  0.973451
6        occupation  TVComplement  0.650569
7      relationship  TVComplement  0.967256
8              race  TVComplement  0.997181
9            gender  TVComplement  0.989833
10     capital-gain  KSComplement  0.917053
11     capital-loss  KSComplement  0.952576
12   hours-per-week  KSComplement  0.772887
13   native-country  TVComplement  0.176361
14           income  TVComp

In [14]:
fig = qlt_report.get_visualization(property_name='Column Shapes')
fig.show()

In [15]:
fig = qlt_report.get_visualization(property_name='Column Pair Trends')
fig.show()

In [6]:
from sdmetrics.reports.single_table import DiagnosticReport

dx_report = DiagnosticReport()
dx_report.generate(
    real_data = splitter.data[1]['train'],
    synthetic_data = postproc.data,
    metadata = real_data_metadata
)
print(f"\n\n")
print(f"get_score: {dx_report.get_score()}")
print(f"\n\n")
print(dx_report.get_properties())
print(f"\n\n")
print(dx_report.get_details(property_name='Data Validity'))
print(f"\n\n")
print(dx_report.get_details(property_name='Data Structure'))


Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<?, ?it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%



get_score: 1.0



         Property  Score
0   Data Validity    1.0
1  Data Structure    1.0



             Column             Metric  Score
0               age  BoundaryAdherence    1.0
1         workclass  CategoryAdherence    1.0
2            fnlwgt  BoundaryAdherence    1.0
3         education  CategoryAdherence    1.0
4   educational-num  BoundaryAdherence    1.0
5    marital-status  CategoryAdherence    1.0
6        occupation  CategoryAdherence    1.0
7      relationship  CategoryAdherence    1.0
8              race  CategoryAdherence    1.0
9            gender  CategoryAdherence    1.0
10     capital-gain  BoundaryAdherence    1.0
11     capital-loss  BoundaryAdherence    1.0
12   hours-per-week  BoundaryAdherence    1.0
13   native-country  CategoryAdherence    1.0
14           income  CategoryAd